In [ ]:
import cv2
import numpy as np
import os

# Paths
video_path = '/content/in1.mp4'
frame_output_dir = '/content/contact_frames/'
annotated_video_output_path = '/content/annotated_contact_video.avi'

# Create directory for saving frames
os.makedirs(frame_output_dir, exist_ok=True)

# Load video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video. Check the video path.")
    exit()

# Function to check if an object is a ball based on circularity and size
def is_ball(contour):
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    circularity = 4 * np.pi * (area / (perimeter * perimeter))
    return 150 < area < 800 and 0.85 < circularity < 1.15

# Frame counter for unique naming
frame_counter = 0

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Convert to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Detect pitch (green color)
    lower_green = np.array([40, 55, 55])
    upper_green = np.array([85, 255, 255])
    pitch_mask = cv2.inRange(hsv, lower_green, upper_green)
    pitch_mask = cv2.morphologyEx(pitch_mask, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))

    # Detect ball (red color)
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])
    red_mask = cv2.inRange(hsv, lower_red1, upper_red1) | cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.GaussianBlur(red_mask, (7, 7), 0)

    # Find contours in the red mask
    contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Process contours to detect the ball
    for contour in contours:
        if is_ball(contour):
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x), int(y))
            radius = int(radius)

            # Find the bottom point of the ball
            ball_bottom_point = (center[0], center[1] + radius)

            # Check if the ball's bottom point is on the pitch
            if 0 <= ball_bottom_point[1] < frame.shape[0] and 0 <= ball_bottom_point[0] < frame.shape[1]:
                if pitch_mask[ball_bottom_point[1], ball_bottom_point[0]] > 0:
                    # Annotate the frame
                    cv2.circle(frame, center, radius, (0, 255, 0), 2)  # Ball outline
                    cv2.circle(frame, ball_bottom_point, 5, (255, 0, 0), -1)  # Contact point
                    cv2.putText(frame, "Contact Point", (ball_bottom_point[0] + 10, ball_bottom_point[1] + 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                    # Save the annotated frame
                    frame_path = os.path.join(frame_output_dir, f"contact_frame_{frame_counter}.jpg")
                    cv2.imwrite(frame_path, frame)
                    print(f"Saved frame: {frame_path}")

                    frame_counter += 1
                    break  # Process only the first valid ball per frame

cap.release()
print(f"Frame processing complete. Frames saved in {frame_output_dir}")

# ------------------------------------------------------------
# Combine saved frames into a video
# ------------------------------------------------------------

# Get list of saved frames
frame_files = sorted([f for f in os.listdir(frame_output_dir) if f.endswith('.jpg')],
                     key=lambda x: int(x.split('_')[-1].split('.')[0]))

if not frame_files:
    print("Error: No frames found to create a video.")
    exit()

# Read the first frame to get frame properties
first_frame = cv2.imread(os.path.join(frame_output_dir, frame_files[0]))
height, width, layers = first_frame.shape

# Define the video writer
fps = 30  # Frames per second
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(annotated_video_output_path, fourcc, fps, (width, height))

# Write frames to the output video
for frame_file in frame_files:
    frame_path = os.path.join(frame_output_dir, frame_file)
    frame = cv2.imread(frame_path)
    out.write(frame)

out.release()
print(f"Annotated video saved at: {annotated_video_output_path}")


Saved frame: /content/contact_frames/contact_frame_0.jpg
Saved frame: /content/contact_frames/contact_frame_1.jpg
Saved frame: /content/contact_frames/contact_frame_2.jpg
Saved frame: /content/contact_frames/contact_frame_3.jpg
Saved frame: /content/contact_frames/contact_frame_4.jpg
Saved frame: /content/contact_frames/contact_frame_5.jpg
Saved frame: /content/contact_frames/contact_frame_6.jpg
Saved frame: /content/contact_frames/contact_frame_7.jpg
Saved frame: /content/contact_frames/contact_frame_8.jpg
Saved frame: /content/contact_frames/contact_frame_9.jpg
Saved frame: /content/contact_frames/contact_frame_10.jpg
Saved frame: /content/contact_frames/contact_frame_11.jpg
Saved frame: /content/contact_frames/contact_frame_12.jpg
Saved frame: /content/contact_frames/contact_frame_13.jpg
Saved frame: /content/contact_frames/contact_frame_14.jpg
Saved frame: /content/contact_frames/contact_frame_15.jpg
Saved frame: /content/contact_frames/contact_frame_16.jpg
Saved frame: /content/co